In [6]:
import pandas as pd


def purge():
    import torch
    from numba import cuda
    import gc
    from time import sleep
    try:
        cuda.select_device(0)
        cuda.close()
        cuda.select_device(0)
        gc.collect()
        sleep(2)
        torch.cuda.empty_cache()
    except:
        pass
    
    
checkpoints = [
    # "facebook/wav2vec2-large-slavic-voxpopuli-v2",
    # "facebook/wav2vec2-large-960h-lv60-self",
    "classla/wav2vec2-large-slavic-parlaspeech-hr",
]
optimal_epochs = {
    "facebook/wav2vec2-large-960h-lv60-self": 9,
    "facebook/wav2vec2-large-slavic-voxpopuli-v2": 11,
    "classla/wav2vec2-large-slavic-parlaspeech-hr": 7,  
}
from utils import train_model, eval_model
results = []
for checkpoint in checkpoints:
    train_config = dict(
        model_name_or_path = checkpoint,
        TASK = "emotion_optimal_epochs",
        NUM_EPOCH = optimal_epochs.get(checkpoint),
        output_column = "target",
        input_column = "path",
        data_files = {
            "train": "001_train.csv",
            "validation": "001_dev.csv",
        },
        clip_seconds = 10,
    )

    output_dir = train_model(train_config)
    import numpy as np
    for split in "dev test".split():
        from pathlib import Path
        eval_config = dict(
            output_column = "target",
            model_name_or_path= str(list(Path(output_dir).glob("checkpoint-*"))[0]) ,
            eval_file= f"001_{split}.csv"
        )

        y_true, y_pred = eval_model(eval_config)
        results.append({**eval_config, 
                        **train_config,
                        "split": split,
                        "y_true": y_true, 
                        "y_pred": y_pred})
        content = pd.DataFrame(data=results).to_json(None, 
                                           orient="records",
                                           lines=True,
                                           )
        with open("004_results_new.jsonl", "a") as f:
            f.writelines(content)


Using custom data configuration default-ca47a7a6555e75ab
Reusing dataset csv (/home/peterr/.cache/huggingface/datasets/csv/default-ca47a7a6555e75ab/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

A classification problem with 5 classes: ['anger', 'fear', 'happiness', 'neutral', 'sadness']


loading configuration file https://huggingface.co/classla/wav2vec2-large-slavic-parlaspeech-hr/resolve/main/config.json from cache at /home/peterr/.cache/huggingface/transformers/05ba655a8826008bb02f30f10fa5690b3df89e2b74031e1c8f9ead2da53e7173.1df24b662a4164adc8330994f0ed99b3b44c58e2263205e3ddba275170f00570
Model config Wav2Vec2Config {
  "_name_or_path": "classla/wav2vec2-large-slavic-parlaspeech-hr",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduct

The target sampling rate: 16000


loading weights file https://huggingface.co/classla/wav2vec2-large-slavic-parlaspeech-hr/resolve/main/pytorch_model.bin from cache at /home/peterr/.cache/huggingface/transformers/5662ba6d35369662a60e23036a4256ad94fb28c06a295a6241e9f1511e10f93a.cadfc2d38d4fbbc4c3d3ef91b87fbe5b9cc2ce8432d754f9b42ffc0cffba07c4
Some weights of the model checkpoint at classla/wav2vec2-large-slavic-parlaspeech-hr were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weig

Epoch,Training Loss,Validation Loss,Accuracy
1,0.988500,0.953553,0.702970
2,0.551300,1.074658,0.688119
3,0.698500,1.417316,0.683168
4,0.062400,1.418077,0.752475
5,0.025100,1.413316,0.732673
6,0.001400,1.509832,0.737624


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: discrete_emotion_annotation_phase, age, Unnamed: 0, expression, manual_transcription, arousal_annotation_phase, split, gender, utterance_id, speaker_id, path, target, discrete_emotion_collection_phase, valence_annotation_phase. If discrete_emotion_annotation_phase, age, Unnamed: 0, expression, manual_transcription, arousal_annotation_phase, split, gender, utterance_id, speaker_id, path, target, discrete_emotion_collection_phase, valence_annotation_phase are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 202
  Batch size = 2
Saving model checkpoint to models/classla_wav2vec2-large-slavic-parlaspeech-hr_emotion_optimal_epochs_/checkpoint-176
Configuration saved in models/classla_wav2vec2-large-slavic-parlaspeech-hr_emotion_optimal_epochs_/checkpoi